In [1]:
from urllib.parse import urlparse
import subprocess
import json

import pandas as pd
import requests

from helpers import derive_filename_from_url

In [2]:
ccn = "241328"
hpi_url = "https://search.hospitalpriceindex.com/hpi2/hospital/SanfordBagleyMedicalCenter/8387or"

In [3]:
def_id = urlparse(hpi_url).path.split('/')[-1].replace('or', '')
def_id

'8387'

In [4]:
json_data = {
    'defId': 8387,
    'priceStatus': 'published',
    'listName': 'machineRead',
}

resp = requests.post('https://api.hospitalpriceindex.com/itemList/detail', json=json_data)
print(resp)

<Response [200]>


In [5]:
json_dict = resp.json()
result = json_dict.get("result")[0] # TODO: check if results list is not empty
ein = result.get("eiNumber")
ein = ein[:2] + "-" + ein[2:]
ein

'41-1266009'

In [6]:
mrf_url = result.get("extractFile").replace('\\', '/')
mrf_url

'https://goldteamstorage.blob.core.windows.net/hpifilestorage/8387/411266009_HPI-CDM_standardcharges.json'

In [7]:
filename = derive_filename_from_url(mrf_url)
filename

'411266009_HPI-CDM_standardcharges.json'

In [8]:
subprocess.run(["wget", "--no-clobber", mrf_url, "-O", filename])

File ‘411266009_HPI-CDM_standardcharges.json’ already there; not retrieving.


CompletedProcess(args=['wget', '--no-clobber', 'https://goldteamstorage.blob.core.windows.net/hpifilestorage/8387/411266009_HPI-CDM_standardcharges.json', '-O', '411266009_HPI-CDM_standardcharges.json'], returncode=1)

In [9]:
in_f = open(filename, 'r')
json_content = json.load(in_f)
in_f.close()

last_updated = json_content[0].get('lastUpdated')
last_updated

'2023-04-25'

In [10]:
df_in = pd.DataFrame(json_content[0].get("item"))
df_in

,description,Associated_Codes,Avg_Gross_Charge,Payer_Allowed_Amount,Cash_Discount_Price,Deidentified_Min_Allowed,DeIdentified_Max_Allowed,payer,iobSelection
0,AMBULANCE,0540,N/A,N/A,N/A,N/A,N/A,NaN,NaN
1,"AMBULANCE - Advanced Life Support, Emergency T...","A0427,0540",N/A,N/A,N/A,N/A,N/A,NaN,NaN
2,"AMBULANCE - Advanced Life Support, Level Ii","A0433,0540",N/A,N/A,N/A,N/A,N/A,NaN,NaN
3,"AMBULANCE - Advanced Life Support, Non-Emergen...","A0426,0540",N/A,N/A,N/A,N/A,N/A,NaN,NaN
4,"AMBULANCE - Basic Life Support, Emergency Tran...","A0429,0540",N/A,N/A,N/A,N/A,N/A,NaN,NaN
...,...,...,...,...,...,...,...,...,...
27478,X-Ray of Upper Arm,73060,666.0000,N/A,599.4000,356.7000,466.1900,United Healthcare Medicare Advantage,Outpatient
27479,"X-Ray of Upper Leg, 2 or More Views",73552,644.3200,N/A,579.8900,122.0200,532.2800,United Healthcare Medicare Advantage,Outpatient
27480,"X-Ray of Wrist, 2 Views",73100,367.2200,196.0300,330.5000,196.0300,328.6700,United Healthcare Medicare Advantage,Outpatient
27481,"X-Ray of Wrist, 3 or More Views",73110,539.9800,281.0400,485.9800,57.4800,475.7100,United Healthcare Medicare Advantage,Outpatient


In [11]:
df_mid = pd.DataFrame(df_in)
set(df_mid['iobSelection'].to_list())

{'Inpatient', 'Outpatient', nan}

In [12]:
df_mid = df_mid.rename(columns={
    'iobSelection': 'setting',
    'Associated_Codes': 'code',
    'payer': 'payer_name'
})

In [13]:
money_columns = df_mid.columns[2:-2].to_list()
remaining_columns = set(df_mid.columns.to_list()) - set(money_columns)
remaining_columns = list(remaining_columns)
remaining_columns

['payer_name', 'setting', 'description', 'code']

In [14]:
df_mid = pd.melt(df_mid, id_vars=remaining_columns, var_name='payer_name2', value_name='standard_charge')
df_mid

,payer_name,setting,description,code,payer_name2,standard_charge
0,NaN,NaN,AMBULANCE,0540,Avg_Gross_Charge,N/A
1,NaN,NaN,"AMBULANCE - Advanced Life Support, Emergency T...","A0427,0540",Avg_Gross_Charge,N/A
2,NaN,NaN,"AMBULANCE - Advanced Life Support, Level Ii","A0433,0540",Avg_Gross_Charge,N/A
3,NaN,NaN,"AMBULANCE - Advanced Life Support, Non-Emergen...","A0426,0540",Avg_Gross_Charge,N/A
4,NaN,NaN,"AMBULANCE - Basic Life Support, Emergency Tran...","A0429,0540",Avg_Gross_Charge,N/A
...,...,...,...,...,...,...
137410,United Healthcare Medicare Advantage,Outpatient,X-Ray of Upper Arm,73060,DeIdentified_Max_Allowed,466.1900
137411,United Healthcare Medicare Advantage,Outpatient,"X-Ray of Upper Leg, 2 or More Views",73552,DeIdentified_Max_Allowed,532.2800
137412,United Healthcare Medicare Advantage,Outpatient,"X-Ray of Wrist, 2 Views",73100,DeIdentified_Max_Allowed,328.6700
137413,United Healthcare Medicare Advantage,Outpatient,"X-Ray of Wrist, 3 or More Views",73110,DeIdentified_Max_Allowed,475.7100


In [15]:
df_mid.loc[df_mid['payer_name2'] != 'Payer_Allowed_Amount', 'payer_name'] = df_mid[df_mid['payer_name2'] != 'Payer_Allowed_Amount']['payer_name2']
del df_mid['payer_name2']

df_mid

,payer_name,setting,description,code,standard_charge
0,Avg_Gross_Charge,NaN,AMBULANCE,0540,N/A
1,Avg_Gross_Charge,NaN,"AMBULANCE - Advanced Life Support, Emergency T...","A0427,0540",N/A
2,Avg_Gross_Charge,NaN,"AMBULANCE - Advanced Life Support, Level Ii","A0433,0540",N/A
3,Avg_Gross_Charge,NaN,"AMBULANCE - Advanced Life Support, Non-Emergen...","A0426,0540",N/A
4,Avg_Gross_Charge,NaN,"AMBULANCE - Basic Life Support, Emergency Tran...","A0429,0540",N/A
...,...,...,...,...,...
137410,DeIdentified_Max_Allowed,Outpatient,X-Ray of Upper Arm,73060,466.1900
137411,DeIdentified_Max_Allowed,Outpatient,"X-Ray of Upper Leg, 2 or More Views",73552,532.2800
137412,DeIdentified_Max_Allowed,Outpatient,"X-Ray of Wrist, 2 Views",73100,328.6700
137413,DeIdentified_Max_Allowed,Outpatient,"X-Ray of Wrist, 3 or More Views",73110,475.7100


In [16]:
df_mid = df_mid[df_mid['standard_charge'] != 'N/A']
df_mid = df_mid[df_mid['standard_charge'].notnull()]
df_mid

,payer_name,setting,description,code,standard_charge
209,Avg_Gross_Charge,Outpatient,11-12 Nerve Conduction Studies,95912,1137.0000
210,Avg_Gross_Charge,Outpatient,12-Lead EKG During Annual Wellness Exam Withou...,G0404,48.0000
212,Avg_Gross_Charge,Outpatient,12-Lead EKG Without Physician Interpretation,93005,258.8200
213,Avg_Gross_Charge,Outpatient,3-4 Nerve Conduction Studies,95908,505.0000
214,Avg_Gross_Charge,Outpatient,5-6 Nerve Conduction Studies,95909,652.0000
...,...,...,...,...,...
137410,DeIdentified_Max_Allowed,Outpatient,X-Ray of Upper Arm,73060,466.1900
137411,DeIdentified_Max_Allowed,Outpatient,"X-Ray of Upper Leg, 2 or More Views",73552,532.2800
137412,DeIdentified_Max_Allowed,Outpatient,"X-Ray of Wrist, 2 Views",73100,328.6700
137413,DeIdentified_Max_Allowed,Outpatient,"X-Ray of Wrist, 3 or More Views",73110,475.7100
